### Getting the files & import statements ###

In [1]:
import pandas as pd
import numpy as np
import os

In [14]:
# Get participant number 
participant_number = input("Enter participant number: ")

base_path = "/Users/alina/Downloads/Thesis/raw/"

# Folder path for the participant
folder_path = os.path.join(base_path, f"Participant {participant_number}")

# Get recording number 
recording_number = input("Enter recording number: ")

# Recording folder path
recording_path = os.path.join(folder_path, f"P{participant_number}_{recording_number}")

# Get files
world_timestamps_file = os.path.join(recording_path, "world_timestamps.csv")
detections_file = os.path.join(recording_path, f"detections_{participant_number}_{recording_number}.csv")
fixations_file = os.path.join(recording_path, "fixations.csv")
face_file = os.path.join(recording_path, "fixations_on_face.csv")

# Load as dfs
world_timestamps_df = pd.read_csv(world_timestamps_file) #contains timestamps for each video frame
detections_df = pd.read_csv(detections_file) #contains detections from YOLO
fixations_df = pd.read_csv(fixations_file) #fixations data
face_df = pd.read_csv(face_file) #fixations on face data from FaceMapper

### Matching video frames and timestamps

In [15]:
## function to see if the gaze is inside the bounding box of an object
def is_gaze_in_bbox(gaze_x, gaze_y, x_min, y_min, x_max, y_max):
    return x_min <= gaze_x <= x_max and y_min <= gaze_y <= y_max

## function to prioritise the smallest object fixated
def filter_objects_by_area(group):
    group = group.copy()
    group["area"] = (group["X_max"] - group["X_min"]) * (group["Y_max"] - group["Y_min"])

    # Find the minimum area
    min_area = group["area"].min()
    
    # Get all objects with the minimum area
    smallest_objects = group[group["area"] == min_area]
    
    if len(smallest_objects) == 1:
        return smallest_objects
    else:
        person_objects = smallest_objects[smallest_objects["Class"] == "person"]
        if not person_objects.empty:
            # Return only one person object (the first one)
            return person_objects.iloc[[0]]

    # If no person among smallest, return just one of the smallest objects
    return smallest_objects.iloc[[0]]

In [16]:
## world_timestamps.csv file from pupil cloud represents timestamps for each video frame
# merge_asof is like a nearest match join for ordered time-series data

# renaming so all timestamp columns are named the same to merge the dfs
fixations_df["timestamp [ns]"] = fixations_df["start timestamp [ns]"]

## making a video_df which contains matching frames and ts
ts = world_timestamps_df['timestamp [ns]']

video_df = pd.DataFrame({
        "Frame": np.arange(len(world_timestamps_df)),
        "timestamp [ns]": ts,
    })

# Make sure everything is sorted by timestamp so merge_asof works properly
fixations_df = fixations_df.sort_values("start timestamp [ns]")
world_timestamps_df = world_timestamps_df.sort_values("timestamp [ns]")

# Merge-asof to assign the closest frame_id to each fixation
fixations_with_frame = pd.merge_asof(
    fixations_df,
    video_df,
    on="timestamp [ns]",
    direction="nearest",  # to match the closest timestamp from video_df, whether it's before or after the fixation timestamp
) 

# merging to assign fixations to frames
fixations_with_detections = pd.merge(
    fixations_with_frame,
    detections_df,
    on="Frame",
    how="left" # keep all the fixations, even if a frame has no detection => nan
)

# calculate in_bbox 
fixations_with_detections["in_bbox"] = fixations_with_detections.apply(
    lambda row: is_gaze_in_bbox(
        row["fixation x [px]"], row["fixation y [px]"],
        row["X_min"], row["Y_min"], row["X_max"], row["Y_max"]
    ),
    axis=1
)

# filter only fixations that match at least one bounding box
in_bbox_fixations = fixations_with_detections[fixations_with_detections["in_bbox"] == True]

#  group by fixation ID and drop "person" if there is another class in same fixation group
filtered_fixations = in_bbox_fixations.groupby(
    "Frame", group_keys=False # grouping on frame as i need fixation id column later
).apply(filter_objects_by_area, include_groups=False)

fixations_on_persons = filtered_fixations.loc[filtered_fixations["Class"] == "person"]
fixations_on_persons.head()

,section id,recording id,fixation id,start timestamp [ns],end timestamp [ns],duration [ms],fixation x [px],fixation y [px],azimuth [deg],elevation [deg],timestamp [ns],X_min,Y_min,X_max,Y_max,Confidence,Class,in_bbox,area
78,ca4ebee2-81a5-4b58-9d74-65a5a2040957,7785da5f-14c2-41eb-bac7-0e9f3e72c4cb,45,1751286234706973112,1751286234791973112,85,1228.516,944.295,27.352941,-20.859393,1751286234706973112,1103.818481,279.505951,1599.100342,1198.958130,0.937119,person,True,4.553880e+05
80,ca4ebee2-81a5-4b58-9d74-65a5a2040957,7785da5f-14c2-41eb-bac7-0e9f3e72c4cb,46,1751286235122347112,1751286235322471112,200,1207.256,723.896,24.851524,-7.267132,1751286235122347112,687.610779,145.729141,1308.902832,1199.335205,0.933464,person,True,6.545971e+05
91,ca4ebee2-81a5-4b58-9d74-65a5a2040957,7785da5f-14c2-41eb-bac7-0e9f3e72c4cb,54,1751286238095091112,1751286238225216112,130,490.040,338.341,-21.827766,16.779506,1751286238095091112,1.273155,0.062180,529.960632,1190.526367,0.840674,person,True,6.293835e+05
92,ca4ebee2-81a5-4b58-9d74-65a5a2040957,7785da5f-14c2-41eb-bac7-0e9f3e72c4cb,55,1751286238720590112,1751286238805715112,85,1090.393,869.815,17.743074,-16.591211,1751286238720590112,1.431656,0.000000,1194.093018,1199.035034,0.873916,person,True,1.430043e+06
93,ca4ebee2-81a5-4b58-9d74-65a5a2040957,7785da5f-14c2-41eb-bac7-0e9f3e72c4cb,56,1751286239025965112,1751286239211089112,185,689.894,710.979,-8.476561,-6.657658,1751286239025965112,233.668518,0.000000,1526.846924,1198.843872,0.671042,person,True,1.550319e+06


### Creating a new file with categorised fixations

In [17]:
# filtering to only fixations on bodies
fixations_on_persons = fixations_on_persons[fixations_on_persons["in_bbox"] == True]

# initialising a df for all types of fixations
categorized_fixations_all = []

# using FaceMapper file to get all the basic fixations information
for _, row in face_df.iterrows():
    timestamp = row['start timestamp [ns]']
    fixation_id = row['fixation id']
    fix_duration = (row['end timestamp [ns]'] - row['start timestamp [ns]'])/1000000     #converting to ms
    fixation_type = "background" #make default type

    # face fixations
    if row.get('fixation on face') == True:
        fixation_type = "face"

    # body fixations (only if not already labeled as face)
    elif not fixations_on_persons[fixations_on_persons['timestamp [ns]'] == timestamp].empty:
        fixation_type = "body"
        matching_row = fixations_on_persons[fixations_on_persons['timestamp [ns]'] == timestamp].iloc[0]
        fixation_id = matching_row['fixation id']
        fix_duration = matching_row['duration [ms]']

    categorized_fixations_all.append({
        'timestamp [ns]': timestamp,
        'type': fixation_type,
        'fixation id': fixation_id,
        'duration [ms]': fix_duration
    })

# saving a file with fixations categorised by face, body and background
all_categorized_fixations_df = pd.DataFrame(categorized_fixations_all)
saving_path = os.path.join(recording_path, f"fixations_on_everything_{participant_number}_{recording_number}.csv")
all_categorized_fixations_df.to_csv(saving_path, index=False)

all_categorized_fixations_df.head()

,timestamp [ns],type,fixation id,duration [ms]
0,1751286209773769112,background,1,2332.120
1,1751286212166014112,background,2,135.124
2,1751286212466263112,background,3,550.499
3,1751286213427136112,background,4,275.250
4,1751286214333010112,background,5,100.125
